In [132]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [133]:
def divided_differences(points, condition):
    """
    points: list of (x_i, y_i) with x_i strictly increasing
    condition: 1 -> forward (first elements of each column)
               0 -> backward (last elements of each column)
    returns: list of selected divided differences (length = len(points))
    """
    x = np.array([p[0] for p in points], dtype=float)
    y = np.array([p[1] for p in points], dtype=float)
    m = len(points)
    if m == 0:
        return []
    if not np.all(np.diff(x) > 0):
        raise ValueError("x values must be strictly increasing.")
    table = np.full((m, m), np.nan, dtype=float)
    table[:, 0] = y.copy()
    for j in range(1, m):
        for i in range(0, m - j):
            table[i, j] = (table[i+1, j-1] - table[i, j-1]) / (x[i+j] - x[i])

    # build DataFrame for display
    data = {'x_i': x, 'y_i': table[:, 0]}
    for j in range(1, m):
        col_vals = [table[i, j] if i < m - j else np.nan for i in range(m)]
        data[f'Order {j}'] = col_vals
    df = pd.DataFrame(data)

    # extract forward or backward selection
    result = []
    for j in range(m):
        col = table[:m - j, j]
        result.append(col[0] if condition == 1 else col[-1])
    return df, result

In [134]:
points = [(1.2, 0.7247), (1.5, 0.1415), (1.7, -0.2577), (1.9, -0.6466), (2.1, -1.0097), (2.4, -1.4748)]

df, result = divided_differences(points, condition = 1)

df.style

,x_i,y_i,Order 1,Order 2,Order 3,Order 4,Order 5
0,1.200000,0.724700,-1.944000,-0.104000,0.332500,-0.010648,-0.015212
1,1.500000,0.141500,-1.996000,0.128750,0.322917,-0.028902,nan
2,1.700000,-0.257700,-1.944500,0.322500,0.296905,nan,nan
3,1.900000,-0.646600,-1.815500,0.530333,nan,nan,nan
4,2.100000,-1.009700,-1.550333,nan,nan,nan,nan
5,2.400000,-1.474800,nan,nan,nan,nan,nan


In [135]:
print(result)

[np.float64(0.7247), np.float64(-1.944), np.float64(-0.10400000000000098), np.float64(0.33250000000000185), np.float64(-0.010648148148143766), np.float64(-0.01521164021166039)]


# Newton Interpolation

## Algorithm


- Divided Differences (Coefficients $\mathbf{D}$)

Given $m+1$ points $\{(x_i, y_i)\}_{i=0}^m$:

1.  **Compute $j$-th order difference:**
    $$f[x_i, \dots, x_{i+j}] = \frac{f[x_{i+1}, \dots, x_{i+j}] - f[x_i, \dots, x_{i+j-1}]}{x_{i+j} - x_i} \quad \text{for } j \ge 1$$
2.  **Base Case (0-th order):**
    $$f[x_i] = y_i$$
3.  **Select Interpolation Coefficients ($\mathbf{D}$):**
    * **Forward Newton (Condition 1):**
        $$D_i = f[x_0, x_1, \dots, x_i] \quad \text{for } i=0, \dots, m$$
    * **Backward Newton (Condition 0):**
        $$D_i = f[x_{m-i}, x_{m-i+1}, \dots, x_m] \quad \text{for } i=0, \dots, m$$

- Polynomial Construction

The Newton interpolation polynomial $P_m(x)$ is given by:
$$P_m(x) = \sum_{i=0}^{m} D_i \cdot B_i(x)$$
where $B_i(x)$ is the basis polynomial.

1.  **Basis Polynomial $B_i(x)$:**
    * **Forward Form:** The nodes used are $x_0, x_1, \dots, x_{i-1}$.
        $$B_0(x) = 1$$
        $$B_i(x) = \prod_{k=0}^{i-1} (x - x_k) \quad \text{for } i \ge 1$$
    * **Backward Form:** The nodes used are $x_m, x_{m-1}, \dots, x_{m-(i-1)}$.
        $$B_0(x) = 1$$
        $$B_i(x) = \prod_{k=0}^{i-1} (x - x_{m-k}) \quad \text{for } i \ge 1$$
        * *(Note: The code handles the backward form by reversing $x$ and $D$, effectively using the forward structure on the reversed data.)*

2.  **Expansion (Coefficient Generation):**
    * Iteratively compute the expanded coefficients of $P_m(x)$ by performing the polynomial multiplication $B_i(x) = (x - x_k) B_{i-1}(x)$ and accumulating the components $D_i B_i(x)$.
    * Final result is the array of coefficients $\mathbf{N_{coeff}}$ for the standard form:
        $$P_m(x) = a_0 + a_1 x + a_2 x^2 + \dots + a_m x^m$$

In [136]:
def newton_interpolation(points, condition):
    """
    Build Newton interpolation polynomial coefficients (lowest -> highest).
    Returns numpy array of coefficients [a0, a1, ..., a_n] (constant first).
    """
    m = len(points)
    if m == 0:
        return np.array([])
    x_arr = np.array([p[0] for p in points], dtype=float)
    if not np.all(np.diff(x_arr) > 0):
        raise ValueError("x values must be strictly increasing for the input points.")
    
    # get divided differences (forward or backward)
    tmp, D_list = divided_differences(points, condition=condition)
    # for backward Newton, reverse x and D so loop is same shape
    if condition == 0:
        D_list = D_list[::-1]
        x_arr = x_arr[::-1]
    N_coeff = np.zeros(1, dtype=float)
    steps = []
    for i in range(m):
        D_i = float(D_list[i])
        # build B_{i-1}(x) using lowest-first coefficients
        if i == 0:
            B = np.array([1.0], dtype=float)
        else:
            B = np.array([1.0], dtype=float)
            for k in range(i):
                # (x - x_k) * B  -> x*B - x_k*B
                xB = np.concatenate(([0.0], B))               # x * B (length len(B)+1)
                aB = np.concatenate((x_arr[k] * B, [0.0]))    # x_k * B padded to same length
                B = xB - aB
        N_i = D_i * B
        # add to total polynomial (pad if needed)
        if len(N_coeff) < len(N_i):
            N_coeff = np.pad(N_coeff, (0, len(N_i) - len(N_coeff)), constant_values=0.0)
        N_coeff[:len(N_i)] += N_i
        steps.append({
            'i': i,
            'D_i': D_i,
            'B_(i-1) coeffs (low->high)': np.round(B, 8).tolist(),
            'N_i coeffs (low->high)': np.round(N_i, 8).tolist()
        })

    step_pd = pd.DataFrame(steps)   
    coeff_pd = pd.DataFrame({'Degree': list(range(len(N_coeff))), 'Coeff': N_coeff})

    return step_pd, coeff_pd


## Result

In [137]:
points = [(1.2, 0.7247), (1.5, 0.1415), (1.7, -0.2577), (1.9, -0.6466), (2.1, -1.0097), (2.4, -1.4748)]
step_pd, coeff_pd = newton_interpolation(points, condition=1)

In [138]:
step_pd.style

,i,D_i,B_(i-1) coeffs (low->high),N_i coeffs (low->high)
0,0,0.724700,[1.0],[0.7247]
1,1,-1.944000,"[-1.2, 1.0]","[2.3328, -1.944]"
2,2,-0.104000,"[1.8, -2.7, 1.0]","[-0.1872, 0.2808, -0.104]"
3,3,0.332500,"[-3.06, 6.39, -4.4, 1.0]","[-1.01745, 2.124675, -1.463, 0.3325]"
4,4,-0.010648,"[5.814, -15.201, 14.75, -6.3, 1.0]","[-0.06190833, 0.1618625, -0.15706019, 0.06708333, -0.01064815]"
5,5,-0.015212,"[-12.2094, 37.7361, -46.176, 27.98, -8.4, 1.0]","[0.185725, -0.57402798, 0.7024127, -0.42562169, 0.12777778, -0.01521164]"


In [139]:
coeff_pd.style

,Degree,Coeff
0,0,1.976667
1,1,0.049310
2,2,-1.021647
3,3,-0.026038
4,4,0.117130
5,5,-0.015212
